The UCSC mapping seems to be missing some combinations of characters. It looks like the mapping has been trimmed to cater ont to widely used combinations. eg. "බෛ" is missing in the set for "ෛ". At the time of the development of the tool, computation capacity may have been a factor in trimming the mappings.

However, the focus of this tool is to cast a reasonably wide net to capture all characters. Therefore we are generating the mappings to capture all available combinations.

Generation will adhere to the following rules:
* New mappings should include all the mappings in UCSC mappings.
* No new mapping should conflict with old mappings
* Only obviously incorrect combinations will be removed from generated combos (we'll revisit this in case of perf issues)

List of combination sets:
* [Set 1: 'ෛ' eg. 'ෂෛ'](#set_1)
* [Set 2: '්‍රෞ' eg. 'ෂ්‍රෞ'](#set_2)
* [Set 3: '්‍යෝ' eg. 'ෂ්‍යෝ'](#set_3)
* [Set 4: '්‍යො' eg. 'ෂ්‍යො'](#set_4)
* [Set 5: '්‍යෙ' eg. 'ෂ්‍යෙ'](#set_5)
* [Set 6: '්‍රෝ' eg. 'ෂ්‍රෝ'](#set_6)
* [Set 7: '්‍රො' eg. 'ෂ්‍රො'](#set_7)
* [Set 8: '්‍රේ' eg. 'ෂ්‍රේ'](#set_8)
* [Set 9: 'ෞ' eg. 'ෂෞ'](#set_9)
* [Set 10: 'ෝ' eg. 'ෂෝ'](#set_10)
* [Set 11: 'ො' eg. 'ෂො'](#set_11)
* [Set 12: 'ේ' eg. 'ෂේ'](#set_12)
* [Set 13: 'ෙ' eg. 'ෂෙ'](#set_13)
* [Set 14: 'ෲ' eg. 'ෂෲ'](#set_14)
* [Set 15: '්‍රි' eg. 'ෂ්‍රි'](#set_15)
* [Set 16: '්‍රී' eg. 'ෂ්‍රී'](#set_16)
* [Set 17: '්‍රෙ' eg. 'ෂ්‍රෙ'](#set_17)
* [Set 18: 'ර්‍' eg. 'ර්‍ෂ'](#set_18)
* [Set 19: 'ර්‍්‍ය' eg. 'ර්‍ෂ්‍ය'](#set_19)

In [1]:
consonants = {
    "l": "ක",
    "L": "ඛ",
    ".": "ග",
    ">": "ඝ",
    "X": "ඞ",
    "Õ": "ඟ",
    "p": "ච",
    "P": "ඡ",
    "c": "ජ",
    "CO": "ඣ",
    "®": "ඣ",
    "[": "ඤ",
    "{": "ඥ",
    "g": "ට",
    "G": "ඨ",
    "v": "ඩ",
    "V": "ඪ",
    "K": "ණ",
    "~": "ඬ",
    ";": "ත",
    ":": "ථ",
    "o": "ද",
    "O": "ධ",
    "k": "න",
    "|": "ඳ",
    "m": "ප",
    "M": "ඵ",
    "n": "බ",
    "N": "භ",
    "u": "ම",
    "U": "ඹ",
    "h": "ය",
    "r": "ර",
    ",": "ල",
    "j": "ව",
    "Y": "ශ",
    "I": "ෂ",
    "i": "ස",
    "y": "හ",
    "<": "ළ",
    "¿": "ළු",
    "*": "ෆ",
}
additional_consonants = {
    "CI": "ක්‍ෂ",
    "Cj": "ක්‍ව",
    "Ë": "ක්‍ෂ",
    "†": "ත්‍ථ",
    "…": "ත්‍ව",
    "‡": "න්‍ද",
    "JO": "න්‍ධ",
}
extra_mappings = {"`.": "ඟ", "`P": "ඦ", "`v": "ඬ", "`o": "ඳ"}
other_letters = {"„": "ද්‍ව", "`j": "ද්‍ව", "Š": "ද්‍ධ", "`O": "ද්‍ධ", "`G": "ට්‍ඨ"}
vowels = {
    "w": "අ",
    "wd": "ආ",
    "we": "ඇ",
    "wE": "ඈ",
    "b": "ඉ",
    "B": "ඊ",
    "W": "උ",
    "W!": "ඌ",
    "R": "ඍ",
    "RD": "ඎ",
    "Ì": "ඏ",
    "Ï": "ඐ",
    "t": "එ",
    "ta": "ඒ",
    "ft": "ඓ",
    "T": "ඔ",
    "´": "ඕ",
    "T!": "ඖ",
}
modifiers = {
    "%s": "්‍රි",
    "%S": "්‍රී",
    "H": "්‍ය",
    "%": "්‍ර",
    "e": "ැ",
    "E": "ෑ",
    "q": "ු",
    "Q": "ූ",
    "s": "ි",
    "S": "ී",
    "!": "ෟ",
    "d": "ා",
    "a": "්",
    "x": "ං",
    "#": "ඃ",
    " ’": "ී",  # unsure. Leaving as is
    " ‘": "ි",  # unsure. Leaving as is
}
repeat_modifiers = {"DD": "ෲ", "D": "ෘ", "f": "ෙ", "ff": "ෛ"}

In [2]:
def concat(fm_prefix, fm_suffix, unicode_prefix, unicode_suffix, in_map, out_map):
    for consonant in in_map.items():
        out_map[fm_prefix + consonant[0] + fm_suffix] = (
            unicode_prefix + consonant[1] + unicode_suffix
        )


def generator(fm_prefix, fm_suffix, unicode_prefix, unicode_suffix, additional=False):

    out = {}

    concat(fm_prefix, fm_suffix, unicode_prefix, unicode_suffix, consonants, out)
    concat(fm_prefix, fm_suffix, unicode_prefix, unicode_suffix, extra_mappings, out)
    if additional:
        concat(
            fm_prefix,
            fm_suffix,
            unicode_prefix,
            unicode_suffix,
            additional_consonants,
            out,
        )
    return out

In [3]:
def printer(char_set):
    print("{", end="")
    for item in char_set.items():
        print(f"'{item[0]}': '{item[1]}'", end=", ")
    print("}", end="")

<a id='set_1'></a>
Mapping for "ෛ" seems to be missing several consonants. Due to the nature of unicode spec
modifiers needs to come after the letter. So this won't be correctly rendered for any case in the form `ff<letter>` if there is no mapping to replace it with the correct unicode form. Therefore generating the combinations and adding them.

In [4]:
char_set_list = []
set_1 = generator("ff", "", "", "ෛ")

print(set_1)

{'ffl': 'කෛ', 'ffL': 'ඛෛ', 'ff.': 'ගෛ', 'ff>': 'ඝෛ', 'ffX': 'ඞෛ', 'ffÕ': 'ඟෛ', 'ffp': 'චෛ', 'ffP': 'ඡෛ', 'ffc': 'ජෛ', 'ffCO': 'ඣෛ', 'ff®': 'ඣෛ', 'ff[': 'ඤෛ', 'ff{': 'ඥෛ', 'ffg': 'ටෛ', 'ffG': 'ඨෛ', 'ffv': 'ඩෛ', 'ffV': 'ඪෛ', 'ffK': 'ණෛ', 'ff~': 'ඬෛ', 'ff;': 'තෛ', 'ff:': 'ථෛ', 'ffo': 'දෛ', 'ffO': 'ධෛ', 'ffk': 'නෛ', 'ff|': 'ඳෛ', 'ffm': 'පෛ', 'ffM': 'ඵෛ', 'ffn': 'බෛ', 'ffN': 'භෛ', 'ffu': 'මෛ', 'ffU': 'ඹෛ', 'ffh': 'යෛ', 'ffr': 'රෛ', 'ff,': 'ලෛ', 'ffj': 'වෛ', 'ffY': 'ශෛ', 'ffI': 'ෂෛ', 'ffi': 'සෛ', 'ffy': 'හෛ', 'ff<': 'ළෛ', 'ff¿': 'ළුෛ', 'ff*': 'ෆෛ', 'ff`.': 'ඟෛ', 'ff`P': 'ඦෛ', 'ff`v': 'ඬෛ', 'ff`o': 'ඳෛ'}


Need to remove 'ළුෛ' and add special cases "එෛ" and "ත්‍රෛ"

In [5]:
set_1["fft"] = "එෛ"
set_1["ff;%"] = "ත්‍රෛ"
del set_1["ff¿"]
char_set_list.append(set_1)

<a id='set_2'></a>
Even though "ප්‍රෞ" is the only commonly used combination, no harm in generating for other consonants.

In [6]:
set_2 = generator("f", "%!", "", "්‍රෞ")
printer(set_2)

{'fl%!': 'ක්‍රෞ', 'fL%!': 'ඛ්‍රෞ', 'f.%!': 'ග්‍රෞ', 'f>%!': 'ඝ්‍රෞ', 'fX%!': 'ඞ්‍රෞ', 'fÕ%!': 'ඟ්‍රෞ', 'fp%!': 'ච්‍රෞ', 'fP%!': 'ඡ්‍රෞ', 'fc%!': 'ජ්‍රෞ', 'fCO%!': 'ඣ්‍රෞ', 'f®%!': 'ඣ්‍රෞ', 'f[%!': 'ඤ්‍රෞ', 'f{%!': 'ඥ්‍රෞ', 'fg%!': 'ට්‍රෞ', 'fG%!': 'ඨ්‍රෞ', 'fv%!': 'ඩ්‍රෞ', 'fV%!': 'ඪ්‍රෞ', 'fK%!': 'ණ්‍රෞ', 'f~%!': 'ඬ්‍රෞ', 'f;%!': 'ත්‍රෞ', 'f:%!': 'ථ්‍රෞ', 'fo%!': 'ද්‍රෞ', 'fO%!': 'ධ්‍රෞ', 'fk%!': 'න්‍රෞ', 'f|%!': 'ඳ්‍රෞ', 'fm%!': 'ප්‍රෞ', 'fM%!': 'ඵ්‍රෞ', 'fn%!': 'බ්‍රෞ', 'fN%!': 'භ්‍රෞ', 'fu%!': 'ම්‍රෞ', 'fU%!': 'ඹ්‍රෞ', 'fh%!': 'ය්‍රෞ', 'fr%!': 'ර්‍රෞ', 'f,%!': 'ල්‍රෞ', 'fj%!': 'ව්‍රෞ', 'fY%!': 'ශ්‍රෞ', 'fI%!': 'ෂ්‍රෞ', 'fi%!': 'ස්‍රෞ', 'fy%!': 'හ්‍රෞ', 'f<%!': 'ළ්‍රෞ', 'f¿%!': 'ළු්‍රෞ', 'f*%!': 'ෆ්‍රෞ', 'f`.%!': 'ඟ්‍රෞ', 'f`P%!': 'ඦ්‍රෞ', 'f`v%!': 'ඬ්‍රෞ', 'f`o%!': 'ඳ්‍රෞ', }

Need to remove obviosly incorrect ones: 2 ඣ්‍රෞ, ඤ්‍රෞ, ඥ්‍රෞ, ණ්‍රෞ, ර්‍රෞ, ල්‍රෞ, ළු්‍රෞ

In [7]:
del set_2["fCO%!"]
del set_2["f®%!"]
del set_2["f[%!"]
del set_2["f{%!"]
del set_2["fK%!"]
del set_2["fr%!"]
del set_2["f,%!"]
del set_2["f¿%!"]
char_set_list.append(set_2)

<a id='set_3'></a>
Set 3 is '්‍යෝ' eg. 'ෂ්‍යෝ'.

In [8]:
set_3 = generator("f", "Hda", "", "්‍යෝ", additional=True)
printer(set_3)

{'flHda': 'ක්‍යෝ', 'fLHda': 'ඛ්‍යෝ', 'f.Hda': 'ග්‍යෝ', 'f>Hda': 'ඝ්‍යෝ', 'fXHda': 'ඞ්‍යෝ', 'fÕHda': 'ඟ්‍යෝ', 'fpHda': 'ච්‍යෝ', 'fPHda': 'ඡ්‍යෝ', 'fcHda': 'ජ්‍යෝ', 'fCOHda': 'ඣ්‍යෝ', 'f®Hda': 'ඣ්‍යෝ', 'f[Hda': 'ඤ්‍යෝ', 'f{Hda': 'ඥ්‍යෝ', 'fgHda': 'ට්‍යෝ', 'fGHda': 'ඨ්‍යෝ', 'fvHda': 'ඩ්‍යෝ', 'fVHda': 'ඪ්‍යෝ', 'fKHda': 'ණ්‍යෝ', 'f~Hda': 'ඬ්‍යෝ', 'f;Hda': 'ත්‍යෝ', 'f:Hda': 'ථ්‍යෝ', 'foHda': 'ද්‍යෝ', 'fOHda': 'ධ්‍යෝ', 'fkHda': 'න්‍යෝ', 'f|Hda': 'ඳ්‍යෝ', 'fmHda': 'ප්‍යෝ', 'fMHda': 'ඵ්‍යෝ', 'fnHda': 'බ්‍යෝ', 'fNHda': 'භ්‍යෝ', 'fuHda': 'ම්‍යෝ', 'fUHda': 'ඹ්‍යෝ', 'fhHda': 'ය්‍යෝ', 'frHda': 'ර්‍යෝ', 'f,Hda': 'ල්‍යෝ', 'fjHda': 'ව්‍යෝ', 'fYHda': 'ශ්‍යෝ', 'fIHda': 'ෂ්‍යෝ', 'fiHda': 'ස්‍යෝ', 'fyHda': 'හ්‍යෝ', 'f<Hda': 'ළ්‍යෝ', 'f¿Hda': 'ළු්‍යෝ', 'f*Hda': 'ෆ්‍යෝ', 'f`.Hda': 'ඟ්‍යෝ', 'f`PHda': 'ඦ්‍යෝ', 'f`vHda': 'ඬ්‍යෝ', 'f`oHda': 'ඳ්‍යෝ', 'fCIHda': 'ක්‍ෂ්‍යෝ', 'fCjHda': 'ක්‍ව්‍යෝ', 'fËHda': 'ක්‍ෂ්‍යෝ', 'f†Hda': 'ත්‍ථ්‍යෝ', 'f…Hda': 'ත්‍ව්‍යෝ', 'f‡Hda': 'න්‍ද්‍යෝ', 'fJOHda': 'න්‍ධ්‍යෝ', }

Need to remove ළු්‍යෝ

In [9]:
del set_3["f¿Hda"]
char_set_list.append(set_3)

<a id='set_4'></a>
Set 4 is '්‍යො' eg. 'ෂ්‍යො'.

In [10]:
set_4 = generator("f", "Hd", "", "්‍යො", additional=True)
printer(set_4)

{'flHd': 'ක්‍යො', 'fLHd': 'ඛ්‍යො', 'f.Hd': 'ග්‍යො', 'f>Hd': 'ඝ්‍යො', 'fXHd': 'ඞ්‍යො', 'fÕHd': 'ඟ්‍යො', 'fpHd': 'ච්‍යො', 'fPHd': 'ඡ්‍යො', 'fcHd': 'ජ්‍යො', 'fCOHd': 'ඣ්‍යො', 'f®Hd': 'ඣ්‍යො', 'f[Hd': 'ඤ්‍යො', 'f{Hd': 'ඥ්‍යො', 'fgHd': 'ට්‍යො', 'fGHd': 'ඨ්‍යො', 'fvHd': 'ඩ්‍යො', 'fVHd': 'ඪ්‍යො', 'fKHd': 'ණ්‍යො', 'f~Hd': 'ඬ්‍යො', 'f;Hd': 'ත්‍යො', 'f:Hd': 'ථ්‍යො', 'foHd': 'ද්‍යො', 'fOHd': 'ධ්‍යො', 'fkHd': 'න්‍යො', 'f|Hd': 'ඳ්‍යො', 'fmHd': 'ප්‍යො', 'fMHd': 'ඵ්‍යො', 'fnHd': 'බ්‍යො', 'fNHd': 'භ්‍යො', 'fuHd': 'ම්‍යො', 'fUHd': 'ඹ්‍යො', 'fhHd': 'ය්‍යො', 'frHd': 'ර්‍යො', 'f,Hd': 'ල්‍යො', 'fjHd': 'ව්‍යො', 'fYHd': 'ශ්‍යො', 'fIHd': 'ෂ්‍යො', 'fiHd': 'ස්‍යො', 'fyHd': 'හ්‍යො', 'f<Hd': 'ළ්‍යො', 'f¿Hd': 'ළු්‍යො', 'f*Hd': 'ෆ්‍යො', 'f`.Hd': 'ඟ්‍යො', 'f`PHd': 'ඦ්‍යො', 'f`vHd': 'ඬ්‍යො', 'f`oHd': 'ඳ්‍යො', 'fCIHd': 'ක්‍ෂ්‍යො', 'fCjHd': 'ක්‍ව්‍යො', 'fËHd': 'ක්‍ෂ්‍යො', 'f†Hd': 'ත්‍ථ්‍යො', 'f…Hd': 'ත්‍ව්‍යො', 'f‡Hd': 'න්‍ද්‍යො', 'fJOHd': 'න්‍ධ්‍යො', }

Need to remove ළු්‍යො.

In [11]:
del set_4["f¿Hd"]
char_set_list.append(set_4)

<a id='set_5'></a>
Set 5 is '්‍යෙ' eg. 'ෂ්‍යෙ'.

In [12]:
set_5 = generator("f", "H", "", "්‍යෙ", additional=True)
printer(set_5)

{'flH': 'ක්‍යෙ', 'fLH': 'ඛ්‍යෙ', 'f.H': 'ග්‍යෙ', 'f>H': 'ඝ්‍යෙ', 'fXH': 'ඞ්‍යෙ', 'fÕH': 'ඟ්‍යෙ', 'fpH': 'ච්‍යෙ', 'fPH': 'ඡ්‍යෙ', 'fcH': 'ජ්‍යෙ', 'fCOH': 'ඣ්‍යෙ', 'f®H': 'ඣ්‍යෙ', 'f[H': 'ඤ්‍යෙ', 'f{H': 'ඥ්‍යෙ', 'fgH': 'ට්‍යෙ', 'fGH': 'ඨ්‍යෙ', 'fvH': 'ඩ්‍යෙ', 'fVH': 'ඪ්‍යෙ', 'fKH': 'ණ්‍යෙ', 'f~H': 'ඬ්‍යෙ', 'f;H': 'ත්‍යෙ', 'f:H': 'ථ්‍යෙ', 'foH': 'ද්‍යෙ', 'fOH': 'ධ්‍යෙ', 'fkH': 'න්‍යෙ', 'f|H': 'ඳ්‍යෙ', 'fmH': 'ප්‍යෙ', 'fMH': 'ඵ්‍යෙ', 'fnH': 'බ්‍යෙ', 'fNH': 'භ්‍යෙ', 'fuH': 'ම්‍යෙ', 'fUH': 'ඹ්‍යෙ', 'fhH': 'ය්‍යෙ', 'frH': 'ර්‍යෙ', 'f,H': 'ල්‍යෙ', 'fjH': 'ව්‍යෙ', 'fYH': 'ශ්‍යෙ', 'fIH': 'ෂ්‍යෙ', 'fiH': 'ස්‍යෙ', 'fyH': 'හ්‍යෙ', 'f<H': 'ළ්‍යෙ', 'f¿H': 'ළු්‍යෙ', 'f*H': 'ෆ්‍යෙ', 'f`.H': 'ඟ්‍යෙ', 'f`PH': 'ඦ්‍යෙ', 'f`vH': 'ඬ්‍යෙ', 'f`oH': 'ඳ්‍යෙ', 'fCIH': 'ක්‍ෂ්‍යෙ', 'fCjH': 'ක්‍ව්‍යෙ', 'fËH': 'ක්‍ෂ්‍යෙ', 'f†H': 'ත්‍ථ්‍යෙ', 'f…H': 'ත්‍ව්‍යෙ', 'f‡H': 'න්‍ද්‍යෙ', 'fJOH': 'න්‍ධ්‍යෙ', }

Need to remove ළු්‍යෙ. And adding special case "hH_": "ර්‍ය්‍ය"

In [13]:
del set_5["f¿H"]
char_set_list.append(set_5)

<a id='set_6'></a>
Set 6 is '්‍රෝ' eg. 'ෂ්‍රෝ'.

In [14]:
set_6 = generator("f", "%da", "", "්‍රෝ", additional=True)
printer(set_6)

{'fl%da': 'ක්‍රෝ', 'fL%da': 'ඛ්‍රෝ', 'f.%da': 'ග්‍රෝ', 'f>%da': 'ඝ්‍රෝ', 'fX%da': 'ඞ්‍රෝ', 'fÕ%da': 'ඟ්‍රෝ', 'fp%da': 'ච්‍රෝ', 'fP%da': 'ඡ්‍රෝ', 'fc%da': 'ජ්‍රෝ', 'fCO%da': 'ඣ්‍රෝ', 'f®%da': 'ඣ්‍රෝ', 'f[%da': 'ඤ්‍රෝ', 'f{%da': 'ඥ්‍රෝ', 'fg%da': 'ට්‍රෝ', 'fG%da': 'ඨ්‍රෝ', 'fv%da': 'ඩ්‍රෝ', 'fV%da': 'ඪ්‍රෝ', 'fK%da': 'ණ්‍රෝ', 'f~%da': 'ඬ්‍රෝ', 'f;%da': 'ත්‍රෝ', 'f:%da': 'ථ්‍රෝ', 'fo%da': 'ද්‍රෝ', 'fO%da': 'ධ්‍රෝ', 'fk%da': 'න්‍රෝ', 'f|%da': 'ඳ්‍රෝ', 'fm%da': 'ප්‍රෝ', 'fM%da': 'ඵ්‍රෝ', 'fn%da': 'බ්‍රෝ', 'fN%da': 'භ්‍රෝ', 'fu%da': 'ම්‍රෝ', 'fU%da': 'ඹ්‍රෝ', 'fh%da': 'ය්‍රෝ', 'fr%da': 'ර්‍රෝ', 'f,%da': 'ල්‍රෝ', 'fj%da': 'ව්‍රෝ', 'fY%da': 'ශ්‍රෝ', 'fI%da': 'ෂ්‍රෝ', 'fi%da': 'ස්‍රෝ', 'fy%da': 'හ්‍රෝ', 'f<%da': 'ළ්‍රෝ', 'f¿%da': 'ළු්‍රෝ', 'f*%da': 'ෆ්‍රෝ', 'f`.%da': 'ඟ්‍රෝ', 'f`P%da': 'ඦ්‍රෝ', 'f`v%da': 'ඬ්‍රෝ', 'f`o%da': 'ඳ්‍රෝ', 'fCI%da': 'ක්‍ෂ්‍රෝ', 'fCj%da': 'ක්‍ව්‍රෝ', 'fË%da': 'ක්‍ෂ්‍රෝ', 'f†%da': 'ත්‍ථ්‍රෝ', 'f…%da': 'ත්‍ව්‍රෝ', 'f‡%da': 'න්‍ද්‍රෝ', 'fJO%da': 'න්‍ධ්‍රෝ', }

Need to remove 'ණ්‍රෝ', 'න්‍රෝ', 'ය්‍රෝ', 'ර්‍රෝ', 'ල්‍රෝ', 'ළු්‍රෝ' and add "føda": "ද්‍රෝ"

In [15]:
set_6["føda"] = "ද්‍රෝ"
del set_6["fK%da"]
del set_6["fk%da"]
del set_6["fr%da"]
del set_6["f,%da"]
del set_6["f¿%da"]
char_set_list.append(set_6)

<a id='set_7'></a>
Set 7 is '්‍රො' eg. 'ෂ්‍රො'.

In [16]:
set_7 = generator("f", "%d", "", "්‍රො")
printer(set_7)

{'fl%d': 'ක්‍රො', 'fL%d': 'ඛ්‍රො', 'f.%d': 'ග්‍රො', 'f>%d': 'ඝ්‍රො', 'fX%d': 'ඞ්‍රො', 'fÕ%d': 'ඟ්‍රො', 'fp%d': 'ච්‍රො', 'fP%d': 'ඡ්‍රො', 'fc%d': 'ජ්‍රො', 'fCO%d': 'ඣ්‍රො', 'f®%d': 'ඣ්‍රො', 'f[%d': 'ඤ්‍රො', 'f{%d': 'ඥ්‍රො', 'fg%d': 'ට්‍රො', 'fG%d': 'ඨ්‍රො', 'fv%d': 'ඩ්‍රො', 'fV%d': 'ඪ්‍රො', 'fK%d': 'ණ්‍රො', 'f~%d': 'ඬ්‍රො', 'f;%d': 'ත්‍රො', 'f:%d': 'ථ්‍රො', 'fo%d': 'ද්‍රො', 'fO%d': 'ධ්‍රො', 'fk%d': 'න්‍රො', 'f|%d': 'ඳ්‍රො', 'fm%d': 'ප්‍රො', 'fM%d': 'ඵ්‍රො', 'fn%d': 'බ්‍රො', 'fN%d': 'භ්‍රො', 'fu%d': 'ම්‍රො', 'fU%d': 'ඹ්‍රො', 'fh%d': 'ය්‍රො', 'fr%d': 'ර්‍රො', 'f,%d': 'ල්‍රො', 'fj%d': 'ව්‍රො', 'fY%d': 'ශ්‍රො', 'fI%d': 'ෂ්‍රො', 'fi%d': 'ස්‍රො', 'fy%d': 'හ්‍රො', 'f<%d': 'ළ්‍රො', 'f¿%d': 'ළු්‍රො', 'f*%d': 'ෆ්‍රො', 'f`.%d': 'ඟ්‍රො', 'f`P%d': 'ඦ්‍රො', 'f`v%d': 'ඬ්‍රො', 'f`o%d': 'ඳ්‍රො', }

Need to remove 'ණ්‍රො', 'න්‍රො', 'ය්‍රො', 'ර්‍රො', 'ල්‍රො', 'ළු්‍රො' and add "fød": "ද්‍රො"

In [17]:
set_7["fød"] = "ද්‍රො"
del set_7["fK%d"]
del set_7["fk%d"]
del set_7["fh%d"]
del set_7["fr%d"]
del set_7["f,%d"]
del set_7["f¿%d"]
char_set_list.append(set_7)

<a id='set_8'></a>
Set 8 is '්‍රේ' eg. 'ෂ්‍රේ'. These mapping are paired with normalization rules derieved in prep. However, this is a special set since the round characters such as 'ම' and 'ට' have a different set of ASCII letters to represent the 'hal' forms. We'll be generating the 'hal' forms with 'a' for all characters to accommodate typing errors and the will add the round 'hal' characters separately.

In [18]:
set_8 = generator("f", "a%", "", "්‍රේ", additional=True)
printer(set_8)

{'fla%': 'ක්‍රේ', 'fLa%': 'ඛ්‍රේ', 'f.a%': 'ග්‍රේ', 'f>a%': 'ඝ්‍රේ', 'fXa%': 'ඞ්‍රේ', 'fÕa%': 'ඟ්‍රේ', 'fpa%': 'ච්‍රේ', 'fPa%': 'ඡ්‍රේ', 'fca%': 'ජ්‍රේ', 'fCOa%': 'ඣ්‍රේ', 'f®a%': 'ඣ්‍රේ', 'f[a%': 'ඤ්‍රේ', 'f{a%': 'ඥ්‍රේ', 'fga%': 'ට්‍රේ', 'fGa%': 'ඨ්‍රේ', 'fva%': 'ඩ්‍රේ', 'fVa%': 'ඪ්‍රේ', 'fKa%': 'ණ්‍රේ', 'f~a%': 'ඬ්‍රේ', 'f;a%': 'ත්‍රේ', 'f:a%': 'ථ්‍රේ', 'foa%': 'ද්‍රේ', 'fOa%': 'ධ්‍රේ', 'fka%': 'න්‍රේ', 'f|a%': 'ඳ්‍රේ', 'fma%': 'ප්‍රේ', 'fMa%': 'ඵ්‍රේ', 'fna%': 'බ්‍රේ', 'fNa%': 'භ්‍රේ', 'fua%': 'ම්‍රේ', 'fUa%': 'ඹ්‍රේ', 'fha%': 'ය්‍රේ', 'fra%': 'ර්‍රේ', 'f,a%': 'ල්‍රේ', 'fja%': 'ව්‍රේ', 'fYa%': 'ශ්‍රේ', 'fIa%': 'ෂ්‍රේ', 'fia%': 'ස්‍රේ', 'fya%': 'හ්‍රේ', 'f<a%': 'ළ්‍රේ', 'f¿a%': 'ළු්‍රේ', 'f*a%': 'ෆ්‍රේ', 'f`.a%': 'ඟ්‍රේ', 'f`Pa%': 'ඦ්‍රේ', 'f`va%': 'ඬ්‍රේ', 'f`oa%': 'ඳ්‍රේ', 'fCIa%': 'ක්‍ෂ්‍රේ', 'fCja%': 'ක්‍ව්‍රේ', 'fËa%': 'ක්‍ෂ්‍රේ', 'f†a%': 'ත්‍ථ්‍රේ', 'f…a%': 'ත්‍ව්‍රේ', 'f‡a%': 'න්‍ද්‍රේ', 'fJOa%': 'න්‍ධ්‍රේ', }

Need to remove 'ණ්‍රේ', 'න්‍රේ', 'ය්‍රේ', 'ර්‍රේ', 'ල්‍රේ','ළු්‍රේ' and add "føa": "ද්‍රේ" and special 'hal' forms.

In [19]:
set_8["føa"] = "ද්‍රේ"
set_8["fÄ%"] = "ඛ්‍රේ"
set_8["få%"] = "ඬ්‍රේ"
set_8["fí%"] = "බ්‍රේ"
set_8["fÉ%"] = "ච්‍රේ"
set_8["fâ%"] = "ඩ්‍රේ"
set_8["fï%"] = "ම්‍රේ"
set_8["fÜ%"] = "ට්‍රේ"
set_8["fõ%"] = "ව්‍රේ"
set_8["fè%"] = "ධ්‍රේ"
del set_8["fKa%"]
del set_8["fka%"]
del set_8["fha%"]
del set_8["fra%"]
del set_8["f,a%"]
del set_8["f¿a%"]
char_set_list.append(set_8)

<a id='set_9'></a>
Set 9 is 'ෞ' eg. 'ෂෞ'. 

In [20]:
set_9 = generator("f", "!", "", "ෞ")
printer(set_9)

{'fl!': 'කෞ', 'fL!': 'ඛෞ', 'f.!': 'ගෞ', 'f>!': 'ඝෞ', 'fX!': 'ඞෞ', 'fÕ!': 'ඟෞ', 'fp!': 'චෞ', 'fP!': 'ඡෞ', 'fc!': 'ජෞ', 'fCO!': 'ඣෞ', 'f®!': 'ඣෞ', 'f[!': 'ඤෞ', 'f{!': 'ඥෞ', 'fg!': 'ටෞ', 'fG!': 'ඨෞ', 'fv!': 'ඩෞ', 'fV!': 'ඪෞ', 'fK!': 'ණෞ', 'f~!': 'ඬෞ', 'f;!': 'තෞ', 'f:!': 'ථෞ', 'fo!': 'දෞ', 'fO!': 'ධෞ', 'fk!': 'නෞ', 'f|!': 'ඳෞ', 'fm!': 'පෞ', 'fM!': 'ඵෞ', 'fn!': 'බෞ', 'fN!': 'භෞ', 'fu!': 'මෞ', 'fU!': 'ඹෞ', 'fh!': 'යෞ', 'fr!': 'රෞ', 'f,!': 'ලෞ', 'fj!': 'වෞ', 'fY!': 'ශෞ', 'fI!': 'ෂෞ', 'fi!': 'සෞ', 'fy!': 'හෞ', 'f<!': 'ළෞ', 'f¿!': 'ළුෞ', 'f*!': 'ෆෞ', 'f`.!': 'ඟෞ', 'f`P!': 'ඦෞ', 'f`v!': 'ඬෞ', 'f`o!': 'ඳෞ', }

Need to remove 'ළුෞ'.

In [21]:
del set_9["f¿!"]
char_set_list.append(set_9)

<a id='set_10'></a>
Set 10 is 'ෝ' eg. 'ෂෝ'.

In [22]:
set_10 = generator("f", "da", "", "ෝ")
printer(set_10)

{'flda': 'කෝ', 'fLda': 'ඛෝ', 'f.da': 'ගෝ', 'f>da': 'ඝෝ', 'fXda': 'ඞෝ', 'fÕda': 'ඟෝ', 'fpda': 'චෝ', 'fPda': 'ඡෝ', 'fcda': 'ජෝ', 'fCOda': 'ඣෝ', 'f®da': 'ඣෝ', 'f[da': 'ඤෝ', 'f{da': 'ඥෝ', 'fgda': 'ටෝ', 'fGda': 'ඨෝ', 'fvda': 'ඩෝ', 'fVda': 'ඪෝ', 'fKda': 'ණෝ', 'f~da': 'ඬෝ', 'f;da': 'තෝ', 'f:da': 'ථෝ', 'foda': 'දෝ', 'fOda': 'ධෝ', 'fkda': 'නෝ', 'f|da': 'ඳෝ', 'fmda': 'පෝ', 'fMda': 'ඵෝ', 'fnda': 'බෝ', 'fNda': 'භෝ', 'fuda': 'මෝ', 'fUda': 'ඹෝ', 'fhda': 'යෝ', 'frda': 'රෝ', 'f,da': 'ලෝ', 'fjda': 'වෝ', 'fYda': 'ශෝ', 'fIda': 'ෂෝ', 'fida': 'සෝ', 'fyda': 'හෝ', 'f<da': 'ළෝ', 'f¿da': 'ළුෝ', 'f*da': 'ෆෝ', 'f`.da': 'ඟෝ', 'f`Pda': 'ඦෝ', 'f`vda': 'ඬෝ', 'f`oda': 'ඳෝ', }

Need to remove 'ළුෝ'

In [23]:
del set_10["f¿da"]
char_set_list.append(set_10)

<a id='set_11'></a>
Set 11 is 'ො' eg. 'ෂො'.

In [24]:
set_11 = generator("f", "d", "", "ො")
printer(set_11)

{'fld': 'කො', 'fLd': 'ඛො', 'f.d': 'ගො', 'f>d': 'ඝො', 'fXd': 'ඞො', 'fÕd': 'ඟො', 'fpd': 'චො', 'fPd': 'ඡො', 'fcd': 'ජො', 'fCOd': 'ඣො', 'f®d': 'ඣො', 'f[d': 'ඤො', 'f{d': 'ඥො', 'fgd': 'ටො', 'fGd': 'ඨො', 'fvd': 'ඩො', 'fVd': 'ඪො', 'fKd': 'ණො', 'f~d': 'ඬො', 'f;d': 'තො', 'f:d': 'ථො', 'fod': 'දො', 'fOd': 'ධො', 'fkd': 'නො', 'f|d': 'ඳො', 'fmd': 'පො', 'fMd': 'ඵො', 'fnd': 'බො', 'fNd': 'භො', 'fud': 'මො', 'fUd': 'ඹො', 'fhd': 'යො', 'frd': 'රො', 'f,d': 'ලො', 'fjd': 'වො', 'fYd': 'ශො', 'fId': 'ෂො', 'fid': 'සො', 'fyd': 'හො', 'f<d': 'ළො', 'f¿d': 'ළුො', 'f*d': 'ෆො', 'f`.d': 'ඟො', 'f`Pd': 'ඦො', 'f`vd': 'ඬො', 'f`od': 'ඳො', }

Need to remove 'ළුො'

In [25]:
del set_11["f¿d"]
char_set_list.append(set_11)

<a id='set_12'></a>
Set 12 is 'ේ' eg. 'ෂේ'. However, this is a special set since the round characters such as 'ම' and 'ට' have a different set of ASCII letters to represent the 'hal' forms. We'll be generating the 'hal' forms with 'a' for all characters to accommodate typing errors and the will add the round 'hal' characters separately.

In [26]:
set_12 = generator("f", "a", "", "ේ", additional=True)
printer(set_12)

{'fla': 'කේ', 'fLa': 'ඛේ', 'f.a': 'ගේ', 'f>a': 'ඝේ', 'fXa': 'ඞේ', 'fÕa': 'ඟේ', 'fpa': 'චේ', 'fPa': 'ඡේ', 'fca': 'ජේ', 'fCOa': 'ඣේ', 'f®a': 'ඣේ', 'f[a': 'ඤේ', 'f{a': 'ඥේ', 'fga': 'ටේ', 'fGa': 'ඨේ', 'fva': 'ඩේ', 'fVa': 'ඪේ', 'fKa': 'ණේ', 'f~a': 'ඬේ', 'f;a': 'තේ', 'f:a': 'ථේ', 'foa': 'දේ', 'fOa': 'ධේ', 'fka': 'නේ', 'f|a': 'ඳේ', 'fma': 'පේ', 'fMa': 'ඵේ', 'fna': 'බේ', 'fNa': 'භේ', 'fua': 'මේ', 'fUa': 'ඹේ', 'fha': 'යේ', 'fra': 'රේ', 'f,a': 'ලේ', 'fja': 'වේ', 'fYa': 'ශේ', 'fIa': 'ෂේ', 'fia': 'සේ', 'fya': 'හේ', 'f<a': 'ළේ', 'f¿a': 'ළුේ', 'f*a': 'ෆේ', 'f`.a': 'ඟේ', 'f`Pa': 'ඦේ', 'f`va': 'ඬේ', 'f`oa': 'ඳේ', 'fCIa': 'ක්‍ෂේ', 'fCja': 'ක්‍වේ', 'fËa': 'ක්‍ෂේ', 'f†a': 'ත්‍ථේ', 'f…a': 'ත්‍වේ', 'f‡a': 'න්‍දේ', 'fJOa': 'න්‍ධේ', }

In [27]:
del set_12["f¿a"]
set_12["fÄ"] = "ඛේ"
set_12["få"] = "ඬේ"
set_12["fí"] = "බේ"
set_12["fÉ"] = "චේ"
set_12["fâ"] = "ඩේ"
set_12["fï"] = "මේ"
set_12["fÜ"] = "ටේ"
set_12["fõ"] = "වේ"
set_12["fè"] = "ධේ"
set_12["fò"] = "ඹේ"
set_12["f¾"] = "රේ"
set_12["fÊ"] = "ජේ"
char_set_list.append(set_12)

<a id='set_13'></a>
Set 13 is 'ෙ' eg. 'ෂෙ'.

In [28]:
set_13 = generator("f", "", "", "ෙ", additional=True)
printer(set_13)

{'fl': 'කෙ', 'fL': 'ඛෙ', 'f.': 'ගෙ', 'f>': 'ඝෙ', 'fX': 'ඞෙ', 'fÕ': 'ඟෙ', 'fp': 'චෙ', 'fP': 'ඡෙ', 'fc': 'ජෙ', 'fCO': 'ඣෙ', 'f®': 'ඣෙ', 'f[': 'ඤෙ', 'f{': 'ඥෙ', 'fg': 'ටෙ', 'fG': 'ඨෙ', 'fv': 'ඩෙ', 'fV': 'ඪෙ', 'fK': 'ණෙ', 'f~': 'ඬෙ', 'f;': 'තෙ', 'f:': 'ථෙ', 'fo': 'දෙ', 'fO': 'ධෙ', 'fk': 'නෙ', 'f|': 'ඳෙ', 'fm': 'පෙ', 'fM': 'ඵෙ', 'fn': 'බෙ', 'fN': 'භෙ', 'fu': 'මෙ', 'fU': 'ඹෙ', 'fh': 'යෙ', 'fr': 'රෙ', 'f,': 'ලෙ', 'fj': 'වෙ', 'fY': 'ශෙ', 'fI': 'ෂෙ', 'fi': 'සෙ', 'fy': 'හෙ', 'f<': 'ළෙ', 'f¿': 'ළුෙ', 'f*': 'ෆෙ', 'f`.': 'ඟෙ', 'f`P': 'ඦෙ', 'f`v': 'ඬෙ', 'f`o': 'ඳෙ', 'fCI': 'ක්‍ෂෙ', 'fCj': 'ක්‍වෙ', 'fË': 'ක්‍ෂෙ', 'f†': 'ත්‍ථෙ', 'f…': 'ත්‍වෙ', 'f‡': 'න්‍දෙ', 'fJO': 'න්‍ධෙ', }

Need to remove ළුෙ

In [29]:
del set_13["f¿"]
char_set_list.append(set_13)

<a id='set_14'></a>
Set 14 is 'ෲ' eg. 'ෂෲ'.

In [30]:
set_14 = generator("", "DD", "", "ෲ")
printer(set_14)

{'lDD': 'කෲ', 'LDD': 'ඛෲ', '.DD': 'ගෲ', '>DD': 'ඝෲ', 'XDD': 'ඞෲ', 'ÕDD': 'ඟෲ', 'pDD': 'චෲ', 'PDD': 'ඡෲ', 'cDD': 'ජෲ', 'CODD': 'ඣෲ', '®DD': 'ඣෲ', '[DD': 'ඤෲ', '{DD': 'ඥෲ', 'gDD': 'ටෲ', 'GDD': 'ඨෲ', 'vDD': 'ඩෲ', 'VDD': 'ඪෲ', 'KDD': 'ණෲ', '~DD': 'ඬෲ', ';DD': 'තෲ', ':DD': 'ථෲ', 'oDD': 'දෲ', 'ODD': 'ධෲ', 'kDD': 'නෲ', '|DD': 'ඳෲ', 'mDD': 'පෲ', 'MDD': 'ඵෲ', 'nDD': 'බෲ', 'NDD': 'භෲ', 'uDD': 'මෲ', 'UDD': 'ඹෲ', 'hDD': 'යෲ', 'rDD': 'රෲ', ',DD': 'ලෲ', 'jDD': 'වෲ', 'YDD': 'ශෲ', 'IDD': 'ෂෲ', 'iDD': 'සෲ', 'yDD': 'හෲ', '<DD': 'ළෲ', '¿DD': 'ළුෲ', '*DD': 'ෆෲ', '`.DD': 'ඟෲ', '`PDD': 'ඦෲ', '`vDD': 'ඬෲ', '`oDD': 'ඳෲ', }

In [31]:
char_set_list.append(set_14)

<a id='set_15'></a>
Set 15 is '්‍රි' eg. 'ෂ්‍රි'. These mapping are paired with normalization rules derieved in prep.

In [32]:
set_15 = generator("", "%s", "", "්‍රි")
printer(set_15)

{'l%s': 'ක්‍රි', 'L%s': 'ඛ්‍රි', '.%s': 'ග්‍රි', '>%s': 'ඝ්‍රි', 'X%s': 'ඞ්‍රි', 'Õ%s': 'ඟ්‍රි', 'p%s': 'ච්‍රි', 'P%s': 'ඡ්‍රි', 'c%s': 'ජ්‍රි', 'CO%s': 'ඣ්‍රි', '®%s': 'ඣ්‍රි', '[%s': 'ඤ්‍රි', '{%s': 'ඥ්‍රි', 'g%s': 'ට්‍රි', 'G%s': 'ඨ්‍රි', 'v%s': 'ඩ්‍රි', 'V%s': 'ඪ්‍රි', 'K%s': 'ණ්‍රි', '~%s': 'ඬ්‍රි', ';%s': 'ත්‍රි', ':%s': 'ථ්‍රි', 'o%s': 'ද්‍රි', 'O%s': 'ධ්‍රි', 'k%s': 'න්‍රි', '|%s': 'ඳ්‍රි', 'm%s': 'ප්‍රි', 'M%s': 'ඵ්‍රි', 'n%s': 'බ්‍රි', 'N%s': 'භ්‍රි', 'u%s': 'ම්‍රි', 'U%s': 'ඹ්‍රි', 'h%s': 'ය්‍රි', 'r%s': 'ර්‍රි', ',%s': 'ල්‍රි', 'j%s': 'ව්‍රි', 'Y%s': 'ශ්‍රි', 'I%s': 'ෂ්‍රි', 'i%s': 'ස්‍රි', 'y%s': 'හ්‍රි', '<%s': 'ළ්‍රි', '¿%s': 'ළු්‍රි', '*%s': 'ෆ්‍රි', '`.%s': 'ඟ්‍රි', '`P%s': 'ඦ්‍රි', '`v%s': 'ඬ්‍රි', '`o%s': 'ඳ්‍රි', }

Need to remove 'ළු්‍රි'. And need to add the combinations for round letters with 'ispilla': "චි ධි වි බි ටි මි ඩි ඹි ඛි ඬි" 

In [33]:
del set_15["¿%s"]
set_15["Ñ%"] = "ච්‍රි"
set_15["ê%"] = "ධ්‍රි"
set_15["ú%"] = "ව්‍රි"
set_15["ì%"] = "බ්‍රි"
set_15["á%"] = "ට්‍රි"
set_15["ñ%"] = "ම්‍රි"
set_15["ä%"] = "ඩ්‍රි"
set_15["ô%"] = "ඹ්‍රි"
set_15["Å%"] = "ඛ්‍රි"
set_15["ç%"] = "ඬ්‍රි"
char_set_list.append(set_15)

<a id='set_16'></a>
Set 16 is '්‍රී' eg. 'ෂ්‍රී'. These mapping are paired with normalization rules derieved in prep.

In [34]:
set_16 = generator("", "%S", "", "්‍රී")
printer(set_16)

{'l%S': 'ක්‍රී', 'L%S': 'ඛ්‍රී', '.%S': 'ග්‍රී', '>%S': 'ඝ්‍රී', 'X%S': 'ඞ්‍රී', 'Õ%S': 'ඟ්‍රී', 'p%S': 'ච්‍රී', 'P%S': 'ඡ්‍රී', 'c%S': 'ජ්‍රී', 'CO%S': 'ඣ්‍රී', '®%S': 'ඣ්‍රී', '[%S': 'ඤ්‍රී', '{%S': 'ඥ්‍රී', 'g%S': 'ට්‍රී', 'G%S': 'ඨ්‍රී', 'v%S': 'ඩ්‍රී', 'V%S': 'ඪ්‍රී', 'K%S': 'ණ්‍රී', '~%S': 'ඬ්‍රී', ';%S': 'ත්‍රී', ':%S': 'ථ්‍රී', 'o%S': 'ද්‍රී', 'O%S': 'ධ්‍රී', 'k%S': 'න්‍රී', '|%S': 'ඳ්‍රී', 'm%S': 'ප්‍රී', 'M%S': 'ඵ්‍රී', 'n%S': 'බ්‍රී', 'N%S': 'භ්‍රී', 'u%S': 'ම්‍රී', 'U%S': 'ඹ්‍රී', 'h%S': 'ය්‍රී', 'r%S': 'ර්‍රී', ',%S': 'ල්‍රී', 'j%S': 'ව්‍රී', 'Y%S': 'ශ්‍රී', 'I%S': 'ෂ්‍රී', 'i%S': 'ස්‍රී', 'y%S': 'හ්‍රී', '<%S': 'ළ්‍රී', '¿%S': 'ළු්‍රී', '*%S': 'ෆ්‍රී', '`.%S': 'ඟ්‍රී', '`P%S': 'ඦ්‍රී', '`v%S': 'ඬ්‍රී', '`o%S': 'ඳ්‍රී', }

Need to remove 'ළු්‍රී'. And need to add the combinations for round letters with 'ispilla': "චී ධී වී බී ටී මී ඩී ඹී ඛී ඬීි" 

In [35]:
del set_16["¿%S"]
set_16["Ö%"] = "ච්‍රී"
set_16["ë%"] = "ධ්‍රී"
set_16["ù%"] = "ව්‍රී"
set_16["î%"] = "බ්‍රී"
set_16["à%"] = "ට්‍රී"
set_16["ó%"] = "ම්‍රී"
set_16["ã%"] = "ඩ්‍රී"
set_16["ö%"] = "ඹ්‍රී"
set_16["Ç%"] = "ඛ්‍රී"
set_16["é%"] = "ඬ්‍රී"
char_set_list.append(set_16)

<a id='set_17'></a>
Set 17 is '්‍රෙ' eg. 'ෂ්‍රෙ'.

In [36]:
set_17 = generator("f", "%", "", "්‍රෙ")
printer(set_17)

{'fl%': 'ක්‍රෙ', 'fL%': 'ඛ්‍රෙ', 'f.%': 'ග්‍රෙ', 'f>%': 'ඝ්‍රෙ', 'fX%': 'ඞ්‍රෙ', 'fÕ%': 'ඟ්‍රෙ', 'fp%': 'ච්‍රෙ', 'fP%': 'ඡ්‍රෙ', 'fc%': 'ජ්‍රෙ', 'fCO%': 'ඣ්‍රෙ', 'f®%': 'ඣ්‍රෙ', 'f[%': 'ඤ්‍රෙ', 'f{%': 'ඥ්‍රෙ', 'fg%': 'ට්‍රෙ', 'fG%': 'ඨ්‍රෙ', 'fv%': 'ඩ්‍රෙ', 'fV%': 'ඪ්‍රෙ', 'fK%': 'ණ්‍රෙ', 'f~%': 'ඬ්‍රෙ', 'f;%': 'ත්‍රෙ', 'f:%': 'ථ්‍රෙ', 'fo%': 'ද්‍රෙ', 'fO%': 'ධ්‍රෙ', 'fk%': 'න්‍රෙ', 'f|%': 'ඳ්‍රෙ', 'fm%': 'ප්‍රෙ', 'fM%': 'ඵ්‍රෙ', 'fn%': 'බ්‍රෙ', 'fN%': 'භ්‍රෙ', 'fu%': 'ම්‍රෙ', 'fU%': 'ඹ්‍රෙ', 'fh%': 'ය්‍රෙ', 'fr%': 'ර්‍රෙ', 'f,%': 'ල්‍රෙ', 'fj%': 'ව්‍රෙ', 'fY%': 'ශ්‍රෙ', 'fI%': 'ෂ්‍රෙ', 'fi%': 'ස්‍රෙ', 'fy%': 'හ්‍රෙ', 'f<%': 'ළ්‍රෙ', 'f¿%': 'ළු්‍රෙ', 'f*%': 'ෆ්‍රෙ', 'f`.%': 'ඟ්‍රෙ', 'f`P%': 'ඦ්‍රෙ', 'f`v%': 'ඬ්‍රෙ', 'f`o%': 'ඳ්‍රෙ', }

Need to remove ණ්‍රෙ, න්‍රෙ, ය්‍රෙ, ර්‍රෙ, ල්‍රෙ and f¿%. Need to add "fø": "ද්‍රෙ".

In [37]:
set_17["fø"] = "ද්‍රෙ"
del set_17["fK%"]
del set_17["fk%"]
del set_17["fh%"]
del set_17["fr%"]
del set_17["f,%"]
del set_17["f¿%"]
char_set_list.append(set_17)

<a id='set_18'></a>
Set 18 is 'ර්‍' eg. 'ර්‍ෂ'.

In [38]:
set_18 = generator("", "_", "ර්‍", "")
printer(set_18)

{'l_': 'ර්‍ක', 'L_': 'ර්‍ඛ', '._': 'ර්‍ග', '>_': 'ර්‍ඝ', 'X_': 'ර්‍ඞ', 'Õ_': 'ර්‍ඟ', 'p_': 'ර්‍ච', 'P_': 'ර්‍ඡ', 'c_': 'ර්‍ජ', 'CO_': 'ර්‍ඣ', '®_': 'ර්‍ඣ', '[_': 'ර්‍ඤ', '{_': 'ර්‍ඥ', 'g_': 'ර්‍ට', 'G_': 'ර්‍ඨ', 'v_': 'ර්‍ඩ', 'V_': 'ර්‍ඪ', 'K_': 'ර්‍ණ', '~_': 'ර්‍ඬ', ';_': 'ර්‍ත', ':_': 'ර්‍ථ', 'o_': 'ර්‍ද', 'O_': 'ර්‍ධ', 'k_': 'ර්‍න', '|_': 'ර්‍ඳ', 'm_': 'ර්‍ප', 'M_': 'ර්‍ඵ', 'n_': 'ර්‍බ', 'N_': 'ර්‍භ', 'u_': 'ර්‍ම', 'U_': 'ර්‍ඹ', 'h_': 'ර්‍ය', 'r_': 'ර්‍ර', ',_': 'ර්‍ල', 'j_': 'ර්‍ව', 'Y_': 'ර්‍ශ', 'I_': 'ර්‍ෂ', 'i_': 'ර්‍ස', 'y_': 'ර්‍හ', '<_': 'ර්‍ළ', '¿_': 'ර්‍ළු', '*_': 'ර්‍ෆ', '`._': 'ර්‍ඟ', '`P_': 'ර්‍ඦ', '`v_': 'ර්‍ඬ', '`o_': 'ර්‍ඳ', }

Need to remove 'ර්‍ළු', 'ර්‍ර'

In [39]:
del set_18["r_"]
del set_18["¿_"]
char_set_list.append(set_18)

<a id='set_19'></a>
Set 19 is 'ර්‍්‍ය' eg. 'ර්‍ෂ්‍ය'.

In [40]:
set_19 = generator("", "H_", "ර්‍", "්‍ය")
printer(set_19)

{'lH_': 'ර්‍ක්‍ය', 'LH_': 'ර්‍ඛ්‍ය', '.H_': 'ර්‍ග්‍ය', '>H_': 'ර්‍ඝ්‍ය', 'XH_': 'ර්‍ඞ්‍ය', 'ÕH_': 'ර්‍ඟ්‍ය', 'pH_': 'ර්‍ච්‍ය', 'PH_': 'ර්‍ඡ්‍ය', 'cH_': 'ර්‍ජ්‍ය', 'COH_': 'ර්‍ඣ්‍ය', '®H_': 'ර්‍ඣ්‍ය', '[H_': 'ර්‍ඤ්‍ය', '{H_': 'ර්‍ඥ්‍ය', 'gH_': 'ර්‍ට්‍ය', 'GH_': 'ර්‍ඨ්‍ය', 'vH_': 'ර්‍ඩ්‍ය', 'VH_': 'ර්‍ඪ්‍ය', 'KH_': 'ර්‍ණ්‍ය', '~H_': 'ර්‍ඬ්‍ය', ';H_': 'ර්‍ත්‍ය', ':H_': 'ර්‍ථ්‍ය', 'oH_': 'ර්‍ද්‍ය', 'OH_': 'ර්‍ධ්‍ය', 'kH_': 'ර්‍න්‍ය', '|H_': 'ර්‍ඳ්‍ය', 'mH_': 'ර්‍ප්‍ය', 'MH_': 'ර්‍ඵ්‍ය', 'nH_': 'ර්‍බ්‍ය', 'NH_': 'ර්‍භ්‍ය', 'uH_': 'ර්‍ම්‍ය', 'UH_': 'ර්‍ඹ්‍ය', 'hH_': 'ර්‍ය්‍ය', 'rH_': 'ර්‍ර්‍ය', ',H_': 'ර්‍ල්‍ය', 'jH_': 'ර්‍ව්‍ය', 'YH_': 'ර්‍ශ්‍ය', 'IH_': 'ර්‍ෂ්‍ය', 'iH_': 'ර්‍ස්‍ය', 'yH_': 'ර්‍හ්‍ය', '<H_': 'ර්‍ළ්‍ය', '¿H_': 'ර්‍ළු්‍ය', '*H_': 'ර්‍ෆ්‍ය', '`.H_': 'ර්‍ඟ්‍ය', '`PH_': 'ර්‍ඦ්‍ය', '`vH_': 'ර්‍ඬ්‍ය', '`oH_': 'ර්‍ඳ්‍ය', }

Need to remove 'ර්‍ළු්‍ය', 'ර්‍ර්‍ය'

In [41]:
del set_19["rH_"]
del set_19["¿H_"]
char_set_list.append(set_19)

Now let's build the JSON with final mappings.

In [42]:
import json

singles = {}
combos = {}
rules = {
    "%a": "a%",
    "A": "a",
    "=": "q",
    "+": "Q",
    "s%": "%s",
    "S%": "%S",
}  # derived in prep
font_name = "FM Abhaya"


def add_to_json(letter_dict):
    for fm, uni in letter_dict.items():
        if len(fm) == 1:
            if fm not in singles:
                singles[fm] = uni
            else:
                print(f"New pair {fm}:{uni}; Old pair {fm}:{singles[fm]}")
        else:
            if fm not in combos:
                combos[fm] = uni
            else:
                print(f"New pair {fm}:{uni}; Old pair {fm}:{combos[fm]}")

Starting with vowels lets add the predetermined letter sets.

In [43]:
add_to_json(vowels)
add_to_json(consonants)
add_to_json(additional_consonants)
add_to_json(extra_mappings)
add_to_json(other_letters)
add_to_json(modifiers)
add_to_json(repeat_modifiers)

Let's ingest the improved UCSC mappings to help with the further additions.

In [44]:
ucsc_mappings = {}
with open("ucsc_improved.txt", "r", encoding="utf-8") as f:
    line_count = 0
    for line in f.readlines():
        line_count += 1
        fm, uni = line.strip().split(": ")
        fm = fm[1:-1]
        uni = uni[1:-1]
        if fm in ucsc_mappings:
            raise Exception
        else:
            ucsc_mappings[fm] = uni

Now let's go through the extended ASCII characters to see any missing singles. Some of these values will not be used due to normalizing rules and combos, but leaving them to complete the picture.

In [45]:
for fm in [chr(i) for i in range(256)]:
    if fm not in singles and fm in ucsc_mappings:
        singles[fm] = ucsc_mappings[fm]
        print(f"'{fm}': '{ucsc_mappings[fm]}'", end=", ")

'"': ',', '$': '/', '&': ')', ''': '.', '(': ':', ')': '*', '+': 'ූ', '-': '-', '/': 'රැ', '=': 'ු', '?': 'රෑ', '@': '?', 'A': '්', 'C': 'ක්‍', 'F': 'ත්‍', 'J': 'න්‍', 'Z': '’', ']': '%', '^': '(', 'z': '‘', '}': '=', '¡': '•', '¢': 'ඳි', '£': 'ඳී', '¤': '–', '¦': ';', '§': 'දී', '¨': 'ලු', 'ª': 'ඳූ', '«': '×', '¬': '+', '­': '÷', '¯': 'ඣි', '°': 'ඣී', '±': 'දැ', '²': '•', '³': '⋆', 'µ': 'i', '¶': 'v', '¸': 'I', '¹': 'V', 'º': 'X', '½': 'ඃ', '¾': 'ර්', 'À': 'ඨි', 'Á': 'ඨී', 'Â': 'ඡී', 'Ä': 'ඛ්', 'Å': 'ඛි', 'Æ': 'ලූ', 'Ç': 'ඛී', 'È': 'දි', 'É': 'ච්', 'Ê': 'ජ්', 'Í': 'රී', 'Î': 'ඪි', 'Ð': 'ඪී', 'Ñ': 'චි', 'Ò': 'ථී', 'Ó': 'ථි', 'Ô': 'ජී', 'Ö': 'චී', '×': 'ඥ', 'Ø': 'ඤ', 'Ù': 'ඞ්', 'Ú': 'ඵී', 'Ü': 'ට්', 'Ý': 'ඵි', 'Þ': 'දා', 'ß': 'රි', 'à': 'ටී', 'á': 'ටි', 'â': 'ඩ්', 'ã': 'ඞී', 'ä': 'ඩි', 'å': 'ඬ්', 'æ': '!', 'ç': 'ඬි', 'è': 'ධ්', 'é': 'ඬී', 'ê': 'ධි', 'ë': 'ධී', 'ì': 'බි', 'í': 'බ්', 'î': 'බී', 'ï': 'ම්', 'ð': 'ජි', 'ñ': 'මි', 'ò': 'ඹ්', 'ó': 'මී', 'ô': 'ඹි', 'õ': 'ව්', 'ö': 'ඹී', '÷': 'ඳ

It seems like due to the behaviour of Python chr function, some of the character in extended ASCII encoding are not matched when adding ASCII letters. Therefore, those letters need to be picked up separately.

In [46]:
for fm, uni in ucsc_mappings.items():
    if len(fm) == 1 and fm not in singles:
        singles[fm] = uni
        print(f"'{fm}': '{uni}'", end=", ")

'›': 'ශ්‍රී', 'ƒ': 'ඳැ', 'Œ': 'ණී', 'ˉ': 'ඣි', '‚': 'ණි', '“': 'ර්‍ණ', 'ˆ': 'න්‍දා', 'œ': 'ර්‍්‍ය', '˜': '”', '—': '”', '™': '{', '∙': '×', 'š': '}', '•': 'x', '−': '÷', '‹': 'ද්‍ධි', '‰': 'ද්‍වි', 

'˜': '”', '—': '”' seems incorrect. Also it doesn't seem necessary to have two different quotes. Using the common double quote for both cases.

In [47]:
singles['˜'] = "\""
singles['—'] = "\""

Now let's add the generated combos to the mix.

In [48]:
for char_set in char_set_list:
    add_to_json(char_set)

Let's check for any missing mappings compared to UCSC improved.

In [49]:
def check_missing():
    for fm, uni in ucsc_mappings.items():

        if fm not in singles and fm not in combos and fm not in rules:


            print(f"'{fm}': '{ucsc_mappings[fm]}'", end=", ")


check_missing()

'f`yda': 'ඟෝ', 'f\{da': 'ඥෝ', 'f\da': 'ඳෝ', 'f`yd': 'ඟො', 'f`Vd': 'ඬො', 'f\{d': 'ඥො', 'f\d': 'ඳො', 'f`Na': 'ඟේ', 'f`y': 'ඟෙ', 'rE': 'රූ', 're': 'රු', '`y': 'ඟ', '`ÿ': 'ඳු', '`¥': 'ඳූ', '`ê': 'ද්‍ධි', '`ú': 'ද්‍වි', 

No singles are missing. Analysing the combos:

* 'f\`yda': 'ඟෝ' - 'Sanyaka Ha' seems incorrect. Need to verify whether '\`y' was commonly used for ඟ instead of '\`.'
* 'f\{da': 'ඥෝ' - Incorrect. Correct combo is already added.
* 'f\da': 'ඳෝ' - Incorrect. Correct combo is 'f\\a'. **Need to add.**
* 'f\`yd': 'ඟො' - 'Sanyaka Ha'
* 'f\`Vd': 'ඬො' - Incorrect. Correct combo for 'ඬො' is already added. There is no "sanyaka mahaprana da'.
* 'f\{d': 'ඥො' - Incorrect. Correct combo is already added.
* 'f\d': 'ඳො' - Incorrect. Correct combo is 'f\\'. **Need to add.**
* 'f\`Na': 'ඟේ' - 'Sanyaka mahaprana ba' seems incorrect.
* 'f\`y': 'ඟෙ' - 'Sanyaka Ha'
* 'rE': 'රූ' - **Need to add.**
* 're': 'රු' - **Need to add.**
* '\`y': 'ඟ' - 'Sanyaka Ha'
* '\`ÿ': 'ඳු' - **Need to add.**
* '\`¥': 'ඳූ' - **Need to add.**
* '\`ê': 'ද්‍ධි' - **Need to add.**
* '\`ú': 'ද්‍වි' - **Need to add.**

Additionally adding the "`" based sanyaka form for 'È'

In [50]:
combos["f\a"] = "ඳෝ"
combos["f\\"] = "ඳො"
combos["rE"] = "රූ"
combos["re"] = "රු"
combos["`ÿ"] = "ඳු"
combos["`¥"] = "ඳූ"
combos["`ê"] = "ද්‍ධි"
combos["`ú"] = "ද්‍වි"
combos["`È"] = "ඳි"

Let's check the missing letters again. It should only output the combos we intentionally ignored.

In [51]:
check_missing()

'f`yda': 'ඟෝ', 'f\{da': 'ඥෝ', 'f\da': 'ඳෝ', 'f`yd': 'ඟො', 'f`Vd': 'ඬො', 'f\{d': 'ඥො', 'f\d': 'ඳො', 'f`Na': 'ඟේ', 'f`y': 'ඟෙ', '`y': 'ඟ', 

Let's check for conflicts with UCSC mapping.

In [52]:
for fm, uni in ucsc_mappings.items():
    if fm in singles:
        if singles[fm] != uni:
            print(f"Singles => '{fm}': '{uni}' and '{singles[fm]}'")
    elif fm in combos:
        if combos[fm] != uni:
            print(f"Combos => '{fm}': '{uni}' and '{combos[fm]}'")
    elif fm in rules:
        if rules[fm] != uni:
            print(f"Rules => '{fm}': '{uni}' and '{rules[fm]}'")

Combos => 'f>%da': '‍ඝ්‍රෝ' and 'ඝ්‍රෝ'
Singles => '˜': '”' and '"'
Singles => '—': '”' and '"'


In [53]:
print("ඝ්‍රෝ".encode("unicode_escape"), "ඝ්‍රෝ".encode("unicode_escape"))

b'\\u0d9d\\u0dca\\u200d\\u0dbb\\u0ddd' b'\\u0d9d\\u0dca\\u200d\\u0dbb\\u0ddd'


Further analysis of the conflict shows that the UCSC mapping has an additional ZWJ in the Unicode character.

In [54]:
print(f"Total mappings excluding rules: {len(singles) + len(combos)}")

Total mappings excluding rules: 1149


Total mappings are more than twice the number of mappings in UCSC mapping. We'll revist to trim this in case of perf issues.

Let's create the JSON mapping file

In [55]:
mappings = {
    "metadata": {
        "script": "sin",
        "language": "sin",
        "font": {
            "name": font_name,
            "url": "",
            "license": "proprietary",
            "crator": "Pushpananda Ekanayaka",
        },
        "mapping": {
            "name": "fm_abhaya",
            "version": 1.0,
            "author": "paarandika",
            "license": "MIT",
            "url": "https://github.com/akuruAI/Pandukabhaya",
        },
    },
    "letters": {
        "vowels": vowels,
        "consonants": consonants,
        "modifiers": modifiers,
        "repeat_modifiers": repeat_modifiers,
        "additional_consonants": additional_consonants,
        "extra_mappings": extra_mappings,
    },
    "mappings": {"rules": rules, "singles": singles, "combos": combos},
}

with open("../pandukabhaya/mappings/fm_abhaya.json", "w", encoding="utf-8") as f:
    json.dump(mappings, f, ensure_ascii=False, indent=4)